In [1]:
import googlesearch
# https://pypi.org/project/request-boost/ # - A better requests and urllib package.
from request_boost import boosted_requests
from serpapi import GoogleSearch
import justext
import html
# https://tailwindcss.com/

In [2]:
_headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

def get_htmls(links):
    try:
        texts = boosted_requests(links, headers=[_headers] * len(links), timeout=15, max_tries=3, parse_json=False)
    except Exception as e:
        print('get_htmls', str(e), links)
        return []
    texts = list(filter(None, texts))
    return texts


def get_content(text):
    
    paragraphs = justext.justext(text, justext.get_stoplist("English"))

    # title
    try:
        title = text[text.find('<title>') + 7 : text.find('</title>')].replace('\n', ' ')
        title = html.unescape(title).strip()
    except:
        title = ''

    # remove all paragraphs that are not plain text
    contents = []
    for paragraph in paragraphs:
        if not paragraph.is_boilerplate:
            contents.append(paragraph.text.replace('\n', ' '))
    return title, '\n'.join(contents)


In [3]:
def google_search(query):
    # https://pypi.org/project/googlesearch-python/
    gsearch = googlesearch.search(query)

    links = list(gsearch)
    htmls = get_htmls(links)
    results = {'response': []}
    for link, text in zip(links, htmls):
        title, content = get_content(text)
        results['response'].append({'url': link, 'title': title, 'content': content})

    return results



def google_image_serp(query):
    # https://serpapi.com/search-api
    gsearch = GoogleSearch({
        "q": query, 
        "tbm": "isch", 
        "ijn": "0",
        "gl": "us", "safe": "active",
        "api_key": "299af8d20c391660a53e49691eb092d7254e397db362d3d074f42d028d852c9b"
    })
    
    import json
    pages = gsearch.get_dict()['organic_results']
    links = [p['link'] for p in pages]
    htmls = get_htmls(links)
    results = {'response': []}
    for p, text in zip(pages, htmls):
        title, link = p['title'], p['link']
        title, content = get_content(text)
        results['response'].append({'url': link, 'title': title, 'content': content})

    return results

In [4]:
# https://serpapi.com/images-results

In [5]:
gsearch = GoogleSearch({
    "q": "husky", 
    "tbm": "isch", 
    "ijn": "0",
    "gl": "us", "safe": "active",
    # "api_key": "299af8d20c391660a53e49691eb092d7254e397db362d3d074f42d028d852c9b" # Curtis
    "api_key": "f1a7740a057dc2d113013d50fe41e86c24878d0ec1ef2182b6e7b5a492c7f7d0" # Carl
})
    

In [6]:
aa = gsearch.get_dict()

In [7]:
aa["images_results"]

[{'position': 1,
  'thumbnail': 'https://serpapi.com/searches/63643745aa6c95f0a415d4ca/images/709e8e39286d66a81aed4ce0d7a7014b9093b0382274295d0fa061eaafc055a2.jpeg',
  'source': 'britannica.com',
  'title': 'Siberian Husky | Characteristics, Overview, Temperament, & Facts |  Britannica',
  'link': 'https://www.britannica.com/animal/Siberian-husky',
  'original': 'https://cdn.britannica.com/84/232784-050-1769B477/Siberian-Husky-dog.jpg',
  'original_width': 1600,
  'original_height': 1142,
  'is_product': False},
 {'position': 2,
  'thumbnail': 'https://serpapi.com/searches/63643745aa6c95f0a415d4ca/images/709e8e39286d66a891420b19aaa0e9f8a0159239a2a447e724f85c0bb7e164bb.jpeg',
  'source': 'akc.org',
  'title': 'Siberian Husky Dog Breed Information',
  'link': 'https://www.akc.org/dog-breeds/siberian-husky/',
  'original': 'https://www.akc.org/wp-content/uploads/2017/11/Siberian-Husky-standing-outdoors-in-the-winter.jpg',
  'original_width': 729,
  'original_height': 486,
  'is_product': 